In [8]:
import requests
import pandas as pd
import time

# 재무제표 데이터를 데이터프레임으로 만드는 함수
def make_fs_dataframe(code):
    url = 'https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A'+str(code)+'&cID=&MenuYn=Y&ReportGB=&NewMenuID=103&stkGb=701'
    # headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'} 
    page = requests.get(url)
    tables = pd.read_html(page.text)
    # 연간데이터만 사용
    temp_df = tables[0]
    print(temp_df)
    # 매출액, 영업이익, 당기순이익만 저장
    temp_df = temp_df.loc[[0,4,15]]
    temp_df = temp_df.set_index(temp_df.columns[0])
    # 4번째 열까지만 저장
    temp_df = temp_df[temp_df.columns[:4]]
    print(tables[1])
    temp_df2 = tables[2]
    print(temp_df2)
    # 자산, 부채, 자본
    temp_df2 = temp_df2.loc[[0,4,8]]
    temp_df2 = temp_df2.set_index(temp_df2.columns[0])
    
    temp_df3 = tables[4]
    # 영업활동으로 인한 현금 흐름
    temp_df3 = temp_df3.loc[[0]]
    temp_df3 = temp_df3.set_index(temp_df3.columns[0])

    # 데이터프레임을 아래로(행으로) 합치기
    df = pd.concat([temp_df,temp_df2,temp_df3])
    # print(df)
    return df

# # 데이터프레임을 전치행렬로 바꾸는 함수
# def change_df(code, dataframe):
#     for num, col in enumerate(dataframe.columns):
#         temp_df = pd.DataFrame({code:dataframe[col]})
#         temp_df = temp_df.T
#         temp_df.columns = [[col]*len(dataframe), temp_df.columns]
#         if num ==0:
#             total_df = temp_df
#         else:
#             # 데이터프레임을 옆으로(열추가) 붙이기
#             total_df = pd.merge(total_df, temp_df, how='outer', left_index=True, right_index=True)
#     # print(total_df)
#     return total_df


# code = pd.read_excel('complist.xlsx')[['종목코드','종목명']]
# code['종목코드'] = code['종목코드'].map('{:06d}'.format)

# for num, code in enumerate(code['종목코드']):
#     try:
#         print(num, code)
#         time.sleep(1)
#         try:
#             df = make_fs_dataframe(code)
#         except requests.exceptions.Timeout:
#             time.sleep(60)
#             df = make_fs_dataframe(code)
#         fs_df_changed = change_df(code, df)
#         if num == 0:
#             total_fs = fs_df_changed
#         else:
#             total_fs = pd.concat([total_fs, fs_df_changed])
#     except :
#         continue

# # 엑셀데이터로 저장한다
# total_fs.to_excel('stock_table.xlsx')

In [9]:
make_fs_dataframe("035720")

                              IFRS(연결)  2020/12  2021/12  2022/12  2023/12  \
0                                  매출액  40714.0  59105.0  67987.0  75570.0   
1                                 매출원가      NaN      NaN      NaN      NaN   
2                                매출총이익  40714.0  59105.0  67987.0  75570.0   
3                판매비와관리비계산에 참여한 계정 펼치기  36209.0  53226.0  62294.0  70961.0   
4                                 영업이익   4505.0   5879.0   5694.0   4609.0   
5                           영업이익(발표기준)   4505.0   5879.0   5694.0   4609.0   
6                   금융수익계산에 참여한 계정 펼치기   2161.0   1778.0   3428.0   3174.0   
7                   금융원가계산에 참여한 계정 펼치기    727.0   3153.0   4877.0   3084.0   
8                   기타수익계산에 참여한 계정 펼치기   3524.0  17334.0  20135.0   1923.0   
9                   기타비용계산에 참여한 계정 펼치기   5461.0   4072.0  11115.0  23144.0   
10  종속기업,공동지배기업및관계기업관련손익계산에 참여한 계정 펼치기     87.0   5101.0   -576.0     40.0   
11                            세전계속사업이익   4089.0  22867.0  12689.

/var/folders/7h/b2q2zw593x188bfm2myyysrh0000gp/T/ipykernel_5188/4175454906.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(page.text)


,2020/12,2021/12,2022/12,2023/12
IFRS(연결),,,,
매출액,40714.0,59105.0,67987.0,75570.0
영업이익,4505.0,5879.0,5694.0,4609.0
당기순이익,1690.0,16405.0,10670.0,-18167.0
자산,119541.0,227846.0,229635.0,251800.0
부채,45424.0,92162.0,94478.0,113214.0
자본,74118.0,135683.0,135157.0,138586.0
영업활동으로인한현금흐름,9711.0,13066.0,6784.0,13411.0


In [ ]:
def change_stock_code(corp_codes, stock_code):
    # 종목코드와 매칭되는 고유번호 구하기
    df = corp_codes[corp_codes['stock_code'] == stock_code]
    if df.empty:
        pass
    else:
        corp_code = df.iloc[0]['corp_code']
        
    return corp_code

In [ ]:
def opendart_finance(api_key, corp_code, bsns_year, reprt_code, fs_div):

    url = 'https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json'
    params = {
        'crtfc_key': api_key, 
        'corp_code': corp_code,
        'bsns_year': bsns_year,   # 사업년도
        'reprt_code': reprt_code, # "11011": 사업보고서
        'fs_div': fs_div
    }

    r = requests.get(url, params=params)
    jo = json.loads(r.text)
    if jo['status'] == '013':  
        df = pd.DataFrame()
        pass
    else:   
        df = json_normalize(jo, 'list')  
    
    return df

In [37]:
##### 변수설정
stock_code = '128940' # 종목코드 한미약품
corp_code = change_stock_code(corp_codes, stock_code) # 종목코드를 고유번호로 변환
reprt_code = ['11011'] # 분기별 report code
fs_div = 'CFS' # 연결/개별 구분

##### 전체재무제표 수집
total_df = pd.DataFrame()
for bsns_year in range(2015,2021):  # 연도는 range함수로 설정
    bsns_year = str(bsns_year)
    for reprt_code in reprt_codes:
        df = opendart_finance(api_key, corp_code, bsns_year, reprt_code,fs_div)
        if df.empty:
            pass
        else:
            if fs_div == 'CFS':
                df['연결구분'] = '연결'
            if fs_div == 'OFS':
                df['연결구분'] = '개별'
                
            total_df = pd.concat([total_df, df])

##### 계정일반화
account_names = ['매출액','매출채권','영업활동으로인한현금흐름']
account_clean = opendart_account_clean(total_df, stock_code, account_names)

##### 데이터클리닝
clean_data = opendart_clean_data(account_clean)
clean_data = clean_data.dropna(axis=1)

##### 재무비율계산
rateoffinance = opendart_rateoffinance(clean_data,stock_code,account_names)

##### 매출채권회수기간 계산
rateoffinance.loc['매출채권회수기간'] = 365 /rateoffinance.loc['활동성:매출채권회전율_(%)'].apply(pd.to_numeric)
rateoffinance.loc['매출채권회수기간','종목코드'] = None
rateoffinance['종목코드'] = rateoffinance['종목코드'].fillna(method='ffill') 
rateoffinance = rateoffinance.dropna(axis=1)
rateoffinance

KeyboardInterrupt: 

In [ ]:
def account_define(account_name):
    account_id_define = {
        '매출액' : ['ifrs_Revenue','',['매출액','수익(매출액)','영업수익', '매출','수익(매출과지분법손익)',
                    '매출과지분법손익(영업수익)','I. 영업수익','I.영업수익','Ⅰ.영업수익', 'Ⅰ. 매출액',
                    'Ⅰ. 매  출  액','수익','계속영업 매출액', 'Ⅰ.매출액', '매출액(영업수익)',
                    '영업수익(매출액)','매출액(수익)','수익 (매출액)', '매출및지분법손익','매출및 지분법손익',
                    '매출 및 지분법손익', '매출 및 지분법 손익','매출액 및 지분법손익','Ⅰ.매출', 'I.매출액',
                    'I.  매출액','수익(순매출액)','매 출 액','I. 매출액','Ⅰ.매출액(주석3)',
                    '매출액(주석3)','매출액(주석3.23)', 'Ⅰ.매출액(주석3.23)', 'I.매출액(주석3.23)', 'Ⅰ.매출액(주석3.24)',
                    '영업 수익','매출액(매출액)', '매출액(주석4,22,31)', 'I. 매출', '매출액(주석2,23,31,39)',
                    '매출액(주석23,31)','고객과의 계약에서 생기는 수익', 'Ⅰ.수익(매출액)','매  출 액', '매출액(주5)',
                    '매출액.', 'I.매출','Ⅰ. 영업수익'],'CIS','IS'],

        '매출총이익':['ifrs_GrossProfit', '',['매출총이익', 'Ⅲ. 매출총이익','Ⅲ. 매출총이익(금융업영업이익)','매출총이익(손실)',
                     'Ⅲ.매출총이익', '매출총손익','매출 총이익', 'III.매출총이익', '매출총손실', 'III. 매출총이익(손실)', '매출 총이익(손실)',
                     'III. 매출총이익', '영업총이익', '매출총이익(영업수익)', 'Ⅲ. 매출총이익(손실)', '매출이익', '용역매출',
                     '영업수익','영업수익(매출)','영업손실','영업수익(매출액)','영업이익'],'CIS','IS'],  

        '영업이익':['dart_OperatingIncomeLoss','',['영업이익(손실)','영업이익','영업손익','III. 영업이익(손실)','III.영업이익',
                    'Ⅲ.영업이익(손실)','Ⅲ.영업이익','Ⅴ. 영업이익(손실)','V. 영업이익','영업손실','Ⅴ.영업이익','IV.영업이익(손실)',
                    'IV.영업이익', 'Ⅳ.영업이익','V.영업이익','V. 영업손익', 'VI.영업이익(손실)','정상영업손익','Ⅳ. 영업이익(손실)',
                    'III. 영업이익','Ⅴ. 영업이익','V. 영업이익(손실)','IV. 영업이익','V.영업이익(손실)','IV. 영업이익(손실)','반기영업이익',
                    '영업이익 (손실)','Ⅳ. 영업이익','Ⅲ. 영업이익(손실)', '영업손실(이익)','IV.영업손익', 'Ⅳ.영업이익(손실)', 'Ⅴ.영업이익(손실)',
                    '영업이익(비용)'],'CIS','IS'],

        '당기순이익':['ifrs_ProfitLoss','',['당기순이익(손실)','분기순이익(손실)','반기순이익','분기순이익','당기순이익','반기순이익(손실)',
                    '-당기순이익(손실)','당(분)기순이익','당(반)기순이익','당기순손익','V. 당기순이익(손실)','ⅩⅠ. 당기순이익(손실)','반기순손실',
                    '당기순손실','분기순손실','분기순이익(순손실)','당기순이익(순손실)','당분기순이익(손실)','당반기순이익(손실)','계속영업당기순이익(손실)',
                    'Ⅹ.당기순이익(손실)','XII.당기순이익(손실)','XIII.당기순이익(손실)','분기순손익','반기순손익','VI. 당기순이익(손실)',
                    '당3분기순이익','당1분기순이익','3분기 순이익','3분기순이익','당기순이익(순실)','당기순손실의 귀속','순이익(손실)',
                    '당기 순이익','연결당기순이익','연결분기(당기)순이익','연결반기(당기)순이익','연결반기순이익','연결분기순이익','Ⅶ.당기순이익',
                    '당(분)기순이익(손실)','당(반)기순이익(손실)','VIII.당기순이익(손실)','VIII.당기순이익','VIII.반기순이익','당기의 순이익',
                    '계속사업순이익(손실)','총포괄손익','분기기순이익(손실)','연결분기순이익(손실)','연결당기순이익(손실)','당기총포괄손실',
                    'Ⅰ.당기순이익(손실)', 'VIII. 분기순이익', 'VIII. 반기순이익', '연결반기순이익(손실)', 'I.당기순이익(손실)', '당기연결순손익',
                    '당기순손익(손실)','Ⅷ. 당기순이익(손실)','법인세 차감 후 반기순이익(손실)','해외사업환산손익','분(당)기순이익(손실)',
                    '반(당)기순이익(손실)','당기(전기)순이익(손실)', '당기연결순이익','총포괄순이익(손실)','연결계속사업당기순이익(손실)',
                    '포괄손익','1. 당기순이익(손실)', 'VIII. 당기순이익(손실)','당분기순이익','VIII. 당기순이익','Ⅶ.당기순이익(손실)',
                    'XⅡ.당기순이익','XII.당기순이익','VII.당기순이익','Ⅶ.반기순이익','당기(분기)순이익(손실)','당기(반기)순이익(손실)',
                    'VII. 반기순이익(손실)','VII. 당기순이익(손실)','XIII. 당기순이익(손실)','분(반)기순이익(손실)','Ⅵ. 당기순이익',
                    'Ⅴ. 당기순이익','XII. 당기순이익', '분기분이익', '당기손익으로 재분류되지 않는 기타포괄손익','당기기순이익','당기순이익 (손실)',
                    '계속영업이익','계속영업 당기순이익','Ⅵ. 당기순이익(손실)','Ⅰ.당기순이익','XI. 당기순이익(손실)','VI.당기순이익(손실)',
                    'IX. 당기순이익(손실)','당기연결손실','총포괄손익 계','당반기순이익','Ⅶ. 당기순이익(손실)','V. 분기순이익','V. 당기순이익',
                    'V. 반기순이익','반기연결순이익(손실)','포괄손익-당기순이익(손실)','Ⅶ. 당기순이익','Ⅶ. 반기순이익','VII. 당기순이익',
                    'VII. 반기순이익','VII. 분기순이익','IX.당기순이익(손실)','Ⅵ.당기순이익(손실)','연결 당기순이익','VII.당기순이익(손실)',
                    'Ⅸ. 당기순이익','XI.당기순이익(손실)','당기순이익(연결)','분기총포괄손익(손실)','당기연결순이익(손실)','VI.당기순손익',
                    '당기순이익(손실):','3분기순이익(손실)','1분기순이익(손실)','분기(당기)순이익(손실)','당기순손실(이익)','X. 당기순이익(손실)',
                    'X. 당반기순이익(손실)','분순이익(손실)','XII.반기순이익(손실)','연결당기순손실','당(분)기순손실','Ⅸ. 당기순이익(손실)',
                    '당기(분기)순이익','Ⅶ. 반기순이익(손실)','반기순손익(손실)','VIII. 반기순이익(손실)','당순이익(손실)','5.당기순이익',
                    '5. 당기순이익','XⅡ.당기순이익(손실)','Ⅶ.반기순이익(손실)','- 당기순이익(손실)','당기순이익과 연결포괄이익','법인순이익과 연결포괄이익',
                    '연결포괄이익','연결당반기순이익(손실)','연결당분기순이익(손실)','연결당기순(손)익','연결반기순(손)익','연결당기순이익 및 총포괄이익',
                    '총포괄이익','당기순이익 및 포괄이익','분기(당기)순이익','반기(당기)순이익'],'CIS','IS'],
        '자산총계':['ifrs_Assets', '',['자산총계','자  산  총  계','자산 총계','자    산    총    계','자 산 총 계','자  산  계',
                    '자산합계'],'BS','BS'],

        '자본총계':['ifrs_Equity', '',['자본총계','자  본  계','자본계','기말자본','반기말자본','IV. 기말자본','IV.기말자본','당분기말','당반기말','자본 총계',
                    '반기말잔액','분기말잔액','기말','반기말','분기말 자본','자본총계1','분기말자본','당분기말 자본','기말 자본','당반기말자본','분기말','당기말자본',
                    'III.기말자본','기말잔액','당기말','전분기말자본','자    본    총    계','당분기말자본','자  본  총  계','기말금액','2019.03.31(당분기말)',
                    'Ⅲ.기말자본','자 본 총 계','Ⅳ.기말자본', 'V.기말자본','당분기말(2019년9월30일)잔액','당기말(2019년12월31일)잔액',
                    '당분기말(2020년3월31일)잔액','당반기말(2020년6월30일)잔액','자본합계', '3분기말자본','1분기말자본','자기주식의 취득',
                    '2015.12.31 기말자본','IV.당(전)기말','반기말 자본','반기자본','자본에 직접 반영된 소유주와의 거래','2015.12.31(당기말)',
                    '기말자본 잔액','2017년 9월 30일 (분기말)','(당기말)','(당분기말)','(당반기말)','2018.9.30(당분기말)', '기   말','총계', 
                    '전기말','자본의조정',],'BS','BS'],

        '부채총계':['ifrs_Liabilities', '',['부채총계', '부  채  총  계', '부채 총계', '부    채    총    계', '부 채 총 계',
                    '부  채  계'],'BS','BS'],
        
        '매출채권':['ifrs_TradeAndOtherCurrentReceivables', '',['매출채권 및 기타채권','매출채권 및 기타유동채권','매출채권','매출채권및기타채권',
                    '기타유동채권','기타채권','미수금및기타수취채권','기타수취채권','(3) 매출채권 및 기타유동채권','기타수치채권',
                    '매출채권및대출채권','2.매출채권','3.기타 수취 채권','4.기타 수취 채권','매출채권 및 기타수취채권','매출채권및기타수취채권',
                    '매출채권및기타유동채권','3.매출채권과 기타채권','3.매출채권 및 기타유동채권','단기기타채권','미수금및기타채권',
                    '매출채권.','매출채권 및 기타금융자산','매출채권 및 수취채권','단기매출채권','매출채권및상각후원가측정금융자산',
                    '대여금및기타수취채권','단기대여금 및 수취채권','단기대여금및수취채권','2)매출채권및기타채권','단기기타수취채권',
                    '매출채권 및 기타상각후원가측정금융자산','매출채권및기타상각후원가측정금융자산', '4. 매출채권','대여금및수취채권',
                    '대여금 및 수취채권','수취채권','매출채권및단기기타채권','기타유동수취채권','대여금및수취채권(유동)','대여금 및 수취채권(유동)',
                    '매출채권 및 기타','5. 매출채권','매출채권및 기타채권','2. 매출채권 및 기타유동채권','매출채권 및 대출채권',
                    '2.매출채권및기타채권','(3)매출채권','(3)매출채권(주석3.5.12)','⑶ 매출채권(주석3.5.12)','매출채권(주석3.5.12.22.28)',
                    '(3)매출채권(주석3.5.12.22.28)','3.매출채권(주석3.5.13.23.29)','매출채권 및 유동기타채권','매출채권및유동기타채권',
                    '매출채권 및 기타금융채권','매출채권 및기타채권','당기손익인식금융자산','기타단기수취채권','3. 매출채권 및 기타유동채권',
                    '매출채권 및 기타단기채권','매출채권 및 기타상각후금융자산','기타채권(유동)','매출채권및기타채권(주석6,30,31)',
                    '유동매출채권 및 기타채권','3. 매출채권및기타채권','3. 매출채권및기타금융자산','매출채권 및 기타유동자산','4. 매출채권및기타채권',
                    '매출채권(주석2,6,12,19,23)','매출채권(주석2,6,12,19,23,27)','매출채권(주석6,12,19,23)','4. 매출채권 및 기타유동채권',
                    '매출채권 및 유동채권','매출채권외수취채권','미수금 및 기타수취채권','매출채권 및 상각후원가측정금융자산(유동)',
                    '매출채권 및 금융리스채권','단기금융자산','매출채권과기타수취채권','매출채권 및 기타채','매출채권 및 기타유동금융자산',
                    '4.매출채권','3.매출채권','미수금 및 기타채권','매출채권 및 상각후원가측정금융자산','유동성보증금','매출채권(주4,6,9)',
                    '매출채권기타채권','기타수취채권(유동)','매출채권 및 계약자산','유동성장기매출채권','유동성기타채권', '유동성 기타채권',
                    '매출채권및기타채권(유동)','기타수채권','매출채권 및 기타 채권','미수금','기타금융채권','기타금융자산', '당좌자산', 
                    '미수수익', '3. 미수수익'],'BS','BS'],
        

        '영업활동으로인한현금흐름':['ifrs_CashFlowsFromUsedInOperatingActivities', '',['영업활동현금흐름','영업활동으로인한현금흐름','Ⅰ.영업활동으로 인한 현금흐름',
                    '영업활동으로 인한 현금흐름','영업활동에서 창출된 현금흐름', 'I. 영업활동으로인한현금흐름','I. 영업활동으로 인한 현금흐름',
                    'I.영업활동으로 인한 현금흐름', 'Ⅰ. 영업에서 창출된 현금','Ⅰ.영업활동현금흐름', 'Ⅰ. 영업에서 창출된 현금흐름',
                    'Ⅰ. 영업활동현금흐름', 'I. 영업활동현금흐름', '영업활동 현금흐름', 'Ⅰ. 영업활동으로 인한 현금흐름', '영업활동으로인한 현금흐름',
                    'I.영업활동현금흐름', '영업활동으로 인한 순현금흐름', '영업활동으로인한순현금흐름', '영업으로인한 현금흐름', 'Ⅰ. 영업활동 현금흐름',
                    '영업활으로 인한 현금흐름', '영업활동으로 인한현금흐름', '1. 영업활동현금흐름', '영업활동으로인현금흐름', '영업활동으로 창출된 현금흐름',
                    '영업활동으로부터의 현금흐름', 'I.영업활동으로인한현금흐름', '영업활동으로 인한 현금 흐름', '1.영업활동으로 인한 현금흐름',
                    '1.영업활동현금흐름','영업활동으로부터 창출된 현금흐름','영업에서 창출된 현금흐름','영업활동현금흐름(주석33)','I. 영업활동 현금흐름',
                    '영업에서창출된현금흐름','I.영업활동 현금흐름','영업에서 사용된 현금흐름','l.영업활동현금흐름','영업으로 인한 현금흐름',
                    '영업활동순현금흐름','Ⅰ.영업활동으로 인한 현금','영업활동으로 인한 현금흐름1','Ⅰ.영업으로 인한 현금흐름',
                    '영업활동 순현금흐름', '영업활동으로부터 발생한 현금흐름', '영업활동으로 인한 순현금흐름액','계속영업'],'CF','CF'],

                }


    account_id_list = account_id_define[account_name]

    return account_id_list

In [ ]:
# 계정일반화
def opendart_account_clean(finance_df, symbol, account_names):
 
    
    finance_df['구분'] = finance_df['bsns_year'] + finance_df['reprt_code'].replace('11013','/1Q').replace('11012','/2Q').replace('11014','/3Q').replace('11011','/4Q')
    finance_df['계정구분'] = finance_df['연결구분'] +'_'+ finance_df['account_nm']
    finance_df['종목코드'] = symbol
    finance_df.rename(columns = {'rcept_no' : '접수번호', 'reprt_code' : '보고서코드','bsns_year' : '사업연도','corp_code' : '고유번호',
                                'sj_div' : '재무제표구분','sj_nm': '재무제표명','account_id': '계정ID', 'account_nm':'계정명',
                                'account_detail':'계정상세','thstrm_nm' : '당기명', 'thstrm_amount' : '당기금액', 'thstrm_add_amount':'당기누적금액',
                                'frmtrm_nm' : '전기명', 'frmtrm_amount':'전기금액','frmtrm_q_nm':'전기명(분/반기)','frmtrm_q_amount':'전기금액(분/반기)',
                                'frmtrm_add_amount':'전기누적금액','bfefrmtrm_nm':'전전기명','bfefrmtrm_amount':'전전기금액','ord':'계정과목 정렬순서'
                                }, inplace = True)
    finance_df = finance_df.set_index('계정구분')
    finance_df = finance_df.reset_index('계정구분')
    finance_df = finance_df.set_index('연결구분')
    finance_df = finance_df.reset_index('연결구분')
    finance_df = finance_df.set_index('종목코드')
    finance_df = finance_df.reset_index('종목코드')
    finance_df = finance_df.set_index('구분')
    finance_df = finance_df.reset_index('구분')   

    
    
    # 빈변수 준비
    except_lists = []
    k_accounts = []

    opendart_finance_final = pd.DataFrame()

    base_data = finance_df.copy()

    if base_data.empty:
        pass
    else:
        base_data['계정ID'] = base_data['계정ID'].str.replace('-full','')
        base_data['계정명'] = base_data['계정명'].str.replace(' ','')
        base_data['계정명'] = base_data['계정명']
        base_total = pd.DataFrame()



        base_data['임시구분'] =  base_data['연결구분'] + '_'+ base_data['구분']
        base_data['임시구분'] = base_data['임시구분'].str.replace(' ','')
        date_divs = base_data['임시구분'].unique()


        for account_name in account_names:
            account_id = account_define(account_name)
            data = base_data.loc[(base_data['재무제표구분'] == account_id[-1])|(base_data['재무제표구분'] == account_id[-2])]

            account_re1 = []
            check_lists = data.loc[data['재무제표구분'] == account_id[-1],'계정명'].unique()
            for account in account_id[2]:
                if account in check_lists:
                    account_re1.append(account)

            account_re2 = []
            check_lists = data.loc[data['재무제표구분'] == account_id[-2],'계정명'].unique()
            for account in account_id[2]:
                if account in check_lists:
                    account_re2.append(account)                          

            for i in date_divs:                
                base_data_first_account1 = data.loc[(data['임시구분'] == i)&(data['계정ID'] == account_id[0])&(data['재무제표구분'] == account_id[-1])].copy()
                if base_data_first_account1.empty:
                    base_data_first_account2 = data.loc[(data['임시구분'] == i)&(data['계정ID'] == account_id[0])&(data['재무제표구분'] == account_id[-2])].copy()     
                    if base_data_first_account2.empty:
                        base_data_second_account1 = data.loc[(data['임시구분'] == i)&(data['계정ID'] == account_id[1])&(data['재무제표구분'] == account_id[-1])].copy()                       
                        if base_data_second_account1.empty:
                            base_data_second_account2 = data.loc[(data['임시구분'] == i)&(data['계정ID'] == account_id[1])&(data['재무제표구분'] == account_id[-2])].copy()
                            if base_data_second_account2.empty:
                                if len(account_re1) != 0:
                                    for account in account_re1:
                                        base_data_third_account1 = data.loc[(data['임시구분'] == i)&(data['계정명'] == account)&(data['재무제표구분'] == account_id[-1])].copy()
                                        if base_data_third_account1.empty:
                                            for account in account_re2:
                                                base_data_third_account2 = data.loc[(data['임시구분'] == i)&(data['계정명'] == account)&(data['재무제표구분'] == account_id[-2])].copy()
                                                if base_data_third_account2.empty:
                                                    pass
                                                else:
                                                    base_data_third_account2['확정계정'] = account_name
                                                    base_total = base_total.append(base_data_third_account2)  
                                                    break      
                       
                                else:
                                    for account in account_re2:
                                        base_data_third_account2 = data.loc[(data['임시구분'] == i)&(data['계정명'] == account)&(data['재무제표구분'] == account_id[-2])].copy()
                                        if base_data_third_account2.empty:
                                            pass
                                        else:
                                            base_data_third_account2['확정계정'] = account_name
                                            base_total = base_total.append(base_data_third_account2)  
                                            break                                                   
                            else:
                                base_data_second_account2['확정계정'] = account_name
                                base_total = base_total.append(base_data_second_account2)                                                  
                        else:
                            base_data_second_account1['확정계정'] = account_name
                            base_total = base_total.append(base_data_second_account1)    
                    else:
                        base_data_first_account2['확정계정'] = account_name
                        base_total = base_total.append(base_data_first_account2)  
                else:
                    base_data_first_account1['확정계정'] = account_name
                    base_total = base_total.append(base_data_first_account1)      


        # 중복제거           
        base_total = base_total.drop_duplicates(['구분','임시구분','당기금액','전기금액','확정계정']) 


    return base_total

In [ ]:
import pandas as pd
import requests
import xml.etree.ElementTree as ET
import zipfile
import io
import json
from pandas.io.json import json_normalize
import numpy as np
import datetime
try:
    from pandas import json_normalize
except ImportError:
    from pandas.io.json import json_normalize
import OpenDartReader
from bs4 import BeautifulSoup
import FinanceDataReader as fdr
%matplotlib inline
import matplotlib.pyplot as plt
pd.options.display.float_format = '{:.2f}'.format


ImportError: cannot import name 'json_normalize' from 'pandas.io.json' (/Users/nayaj/PycharmProjects/stock-prediction/venv/lib/python3.11/site-packages/pandas/io/json/__init__.py)

In [57]:
import requests
import json
import pandas as pd
from pandas import json_normalize

def change_stock_code(corp_codes, stock_code):
    df = corp_codes[corp_codes['stock_code'] == stock_code]
    if df.empty:
        return None
    else:
        return df.iloc[0]['corp_code']

def opendart_finance(api_key, corp_code, bsns_year, reprt_code, fs_div):
    url = 'https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json'
    params = {
        'crtfc_key': api_key, 
        'corp_code': corp_code,
        'bsns_year': bsns_year,   
        'reprt_code': reprt_code, 
        'fs_div': fs_div
    }

    r = requests.get(url, params=params)
    jo = json.loads(r.text)
    if jo['status'] == '013':  
        return pd.DataFrame()
    else:  
        # print(json_normalize(jo, 'list')  ) 
        return json_normalize(jo, 'list')  
    

def account_define(account_name):
    account_id_define = {
        'Net Income': ['ifrs_ProfitLoss', '', ['당기순이익'], 'CIS', 'IS'],
        'Current Assets': ['ifrs_CurrentAssets', '', ['유동자산'], 'BS', 'BS'],
        'Equity': ['ifrs_Equity', '', ['자본총계'], 'BS', 'BS'],
        'Liabilities': ['ifrs_Liabilities', '', ['부채총계'], 'BS', 'BS'],
        'Market Capitalization': ['market_cap', '', ['시가총액'], 'BS', 'BS']
    }
    return account_id_define.get(account_name, [])

def opendart_account_clean(finance_df, symbol, account_names):
    finance_df['구분'] = finance_df['bsns_year'] + finance_df['reprt_code'].replace('11013', '/1Q').replace('11012', '/2Q').replace('11014', '/3Q').replace('11011', '/4Q')
    finance_df['계정구분'] = finance_df['연결구분'] + '_' + finance_df['account_nm']
    finance_df['종목코드'] = symbol

    finance_df.rename(columns={
        'rcept_no': '접수번호', 'reprt_code': '보고서코드', 'bsns_year': '사업연도', 'corp_code': '고유번호',
        'sj_div': '재무제표구분', 'sj_nm': '재무제표명', 'account_id': '계정ID', 'account_nm': '계정명',
        'account_detail': '계정상세', 'thstrm_nm': '당기명', 'thstrm_amount': '당기금액', 'thstrm_add_amount': '당기누적금액',
        'frmtrm_nm': '전기명', 'frmtrm_amount': '전기금액', 'frmtrm_q_nm': '전기명(분/반기)', 'frmtrm_q_amount': '전기금액(분/반기)',
        'frmtrm_add_amount': '전기누적금액', 'bfefrmtrm_nm': '전전기명', 'bfefrmtrm_amount': '전전기금액', 'ord': '계정과목 정렬순서'
    }, inplace=True)

    finance_df['임시구분'] = finance_df['연결구분'] + '_' + finance_df['구분']
    date_divs = finance_df['임시구분'].unique()

    base_total = pd.DataFrame()
    for account_name in account_names:
        account_id = account_define(account_name)
        if len(account_id) < 4:
            continue
        data = finance_df.loc[
            (finance_df['재무제표구분'] == account_id[3]) | (finance_df['재무제표구분'] == account_id[4])
        ]
        account_re1 = [acc for acc in account_id[2] if acc in data.loc[data['재무제표구분'] == account_id[3], '계정명'].unique()]
        account_re2 = [acc for acc in account_id[2] if acc in data.loc[data['재무제표구분'] == account_id[4], '계정명'].unique()]

        for i in date_divs:
            base_data_account = data.loc[
                (data['임시구분'] == i) & (data['계정ID'] == account_id[0]) & 
                (data['재무제표구분'] == account_id[3])
            ]
            if base_data_account.empty:
                base_data_account = data.loc[
                    (data['임시구분'] == i) & (data['계정ID'] == account_id[0]) & 
                    (data['재무제표구분'] == account_id[4])
                ]
            if base_data_account.empty:
                base_data_account = data.loc[
                    (data['임시구분'] == i) & (data['계정명'].isin(account_re1)) & 
                    (data['재무제표구분'] == account_id[3])
                ]
            if base_data_account.empty:
                base_data_account = data.loc[
                    (data['임시구분'] == i) & (data['계정명'].isin(account_re2)) & 
                    (data['재무제표구분'] == account_id[4])
                ]
            if not base_data_account.empty:
                base_data_account['확정계정'] = account_name
                base_total = pd.concat([base_total, base_data_account])

    base_total = base_total.drop_duplicates(['구분', '임시구분', '당기금액', '전기금액', '확정계정'])
    return base_total


def calculate_financial_ratios(api_key, corp_codes, stock_code, reprt_codes, fs_div, years):
    corp_code = change_stock_code(corp_codes, stock_code)
    if not corp_code:
        return pd.DataFrame()
    
    total_df = pd.DataFrame()
    for bsns_year in years:
        for reprt_code in reprt_codes:
            df = opendart_finance(api_key, corp_code, bsns_year, reprt_code, fs_div)
            if not df.empty:
                df['연결구분'] = '연결' if fs_div == 'CFS' else '개별'
                total_df = pd.concat([total_df, df])

    account_names = ['Net Income', 'Current Assets', 'Equity', 'Liabilities', 'Market Capitalization']
    account_clean = opendart_account_clean(total_df, stock_code, account_names)

    net_income = account_clean[account_clean['확정계정'] == 'Net Income'][['구분', '당기금액']].rename(columns={'당기금액': 'Net Income'})
    current_assets = account_clean[account_clean['확정계정'] == 'Current Assets'][['구분', '당기금액']].rename(columns={'당기금액': 'Current Assets'})
    equity = account_clean[account_clean['확정계정'] == 'Equity'][['구분', '당기금액']].rename(columns={'당기금액': 'Equity'})
    liabilities = account_clean[account_clean['확정계정'] == 'Liabilities'][['구분', '당기금액']].rename(columns={'당기금액': 'Liabilities'})
    market_cap = account_clean[account_clean['확정계정'] == 'Market Capitalization'][['구분', '당기금액']].rename(columns={'당기금액': 'Market Cap'})


    combined_df = pd.merge(net_income, current_assets, on='구분', how='outer')
    combined_df = pd.merge(combined_df, equity, on='구분', how='outer')
    combined_df = pd.merge(combined_df, liabilities, on='구분', how='outer')
    combined_df = pd.merge(combined_df, market_cap, on='구분', how='outer')
    
    # Convert columns to numeric
    combined_df['Net Income'] = pd.to_numeric(combined_df['Net Income'], errors='coerce')
    combined_df['Current Assets'] = pd.to_numeric(combined_df['Current Assets'], errors='coerce')
    combined_df['Equity'] = pd.to_numeric(combined_df['Equity'], errors='coerce')
    combined_df['Liabilities'] = pd.to_numeric(combined_df['Liabilities'], errors='coerce')
    combined_df['Market Cap'] = pd.to_numeric(combined_df['Market Cap'], errors='coerce')


    # Calculate Invested Capital and ROIC
    combined_df['Invested Capital'] = combined_df['Equity'] + combined_df['Liabilities']
    combined_df['ROIC'] = combined_df['Net Income'] / combined_df['Invested Capital']

    return combined_df[['구분', 'Net Income', 'Current Assets', 'Equity', 'Liabilities', 'ROIC', 'Market Cap']]


def opendart_corp_codes(api_key):

    #전체 고유번호 구하기
    url = 'https://opendart.fss.or.kr/api/corpCode.xml'
    params = { 'crtfc_key': api_key, }

    r = requests.get(url, params=params)
    try:
        tree = ET.XML(r.content)
        status = tree.find('status').text
        message = tree.find('message').text
        if status != '000':
            raise ValueError({'status': status, 'message': message})
    except ET.ParseError as e:
        pass

    zf = zipfile.ZipFile(io.BytesIO(r.content))
    xml_data = zf.read('CORPCODE.xml')

    tree = ET.XML(xml_data)
    all_records = []

    element = tree.findall('list')
    for i, child in enumerate(element):
        record = {}
        for i, subchild in enumerate(child):
            record[subchild.tag] = subchild.text
        all_records.append(record)

    corp_codes = pd.DataFrame(all_records)
    
    return corp_codes

SyntaxError: invalid syntax (2468012057.py, line 31)

In [59]:
# Example usage:
# def get
api_key = '7943739912fd8119d6f638525b458d5a2b598835'
corp_codes = opendart_corp_codes(api_key)
stock_code = '035720'
reprt_codes = ['11012','11013','11014','11011']
fs_div = 'CFS'
years = range(2022, 2024)

financial_ratios = calculate_financial_ratios(api_key, corp_codes, stock_code, reprt_codes, fs_div, years)
financial_ratios

/var/folders/7h/b2q2zw593x188bfm2myyysrh0000gp/T/ipykernel_5188/2738167681.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_data_account['확정계정'] = account_name
/var/folders/7h/b2q2zw593x188bfm2myyysrh0000gp/T/ipykernel_5188/2738167681.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_data_account['확정계정'] = account_name
/var/folders/7h/b2q2zw593x188bfm2myyysrh0000gp/T/ipykernel_5188/2738167681.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


,구분,Net Income,Current Assets,Equity,Liabilities,ROIC,Market Cap
0,2022/1Q,NaN,8204175956140,14895766842128,9548025165189,NaN,NaN
1,2022/2Q,NaN,7929367119134,14821152791339,9955798559392,NaN,NaN
2,2022/3Q,NaN,8286537793825,15276812664302,10168406425358,NaN,NaN
3,2022/4Q,NaN,7755732886687,13515682590325,9447968971124,NaN,NaN
4,2023/1Q,NaN,8109598258666,15302267362137,10284110209034,NaN,NaN
5,2023/2Q,NaN,8091538538561,15446812026258,10605089557617,NaN,NaN
6,2023/3Q,NaN,8695272771351,15660696103271,11124868765051,NaN,NaN
7,2023/4Q,NaN,8943259940386,13858599224126,11321369715195,NaN,NaN


In [55]:
financial_ratios

,구분,Net Income,Current Assets,Equity,Liabilities,ROIC,Market Cap
0,2022/4Q,101558192480,694158250909,1009226682083,915359048755,0.052769,NaN
1,2023/4Q,165366177835,730623667543,1100210644155,798457731813,0.087096,NaN


In [44]:
def load_allstock_KRX():
    krx_url = 'https://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13'
    stk_data = pd.read_html(krx_url, header=0, encoding='euc-kr')[0]
    stk_data = stk_data[['회사명', '종목코드']]
    stk_data = stk_data.rename(columns={'회사명': 'Name', '종목코드': 'Code'})

    stk_data['Code'] = stk_data['Code'].apply(lambda input: '0' * (6 - len(str(input))) + str(input))

    return stk_data

In [45]:
stk_data = load_allstock_KRX()

/Users/nayaj/PycharmProjects/stock-prediction/venv/lib/python3.11/site-packages/bs4/__init__.py:228: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


In [46]:
stk_data

,Name,Code
0,BGF리테일,282330
1,CJ제일제당,097950
2,DRB동일,004840
3,E1,017940
4,HDC,012630
...,...,...
2687,타스컴,336040
2688,테크엔,308700
2689,파워풀엑스,266870
2690,펨토바이오메드,327610
